In [1]:
from math import *
import csv

# Orbit determination

<img src="orbit%20det.png" alt="cosine vector" style="width: 500px;"/>

## purpose of the program

The purpose of this program is to find the six orbital elements of an astroid namely:
- The semi-major axis of the orbit.
- The eccentricity of the orbit.
- The inclination of the orbit.
- The longitude of the ascending node.
- The argument of perihelion.
- The true anomaly.

using the data from three seperate observation of the given astroid.

## inputs

The inputs include 
- the time of each obervation.
- the right ascension. 
- the declination of the aseteroid.
- the postion and the velocity vectors relative to earth.

In [2]:
with open('data.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)
    
ndata = []
for i in range(1,10):
    ndata.append(list(map(float, data[i])))

c = 173.1446 #speed of light in
k = 0.01720209895
mu = 398600
data = ndata

In [3]:
# Consider rows 1-3, which contain the data for the first of three observations. 
# This is how the data is organized:

# day      month     year      UT (hrs)  UT (mins)    UT (secs)
# RA (hrs) RA (mins) RA (secs) Dec (deg) Dec (arcmin) Dec (arcsec)
# R [i]    R [j]     R [k]     Rdot [i]  Rdot [j]     Rdot [k]

breaking the data into three different lists:

In [4]:
obs1 = data[:][:3]
obs2 = data[:][3:6]
obs3 = data[:][6:]

In [5]:
def time(data):
    year = float(data[0][2])
    month = float(data[0][1])
    day = data[0][0]
    UT = data[0][3] + data[0][4]/60 + data[0][5]/3600
    J0 = 367*year - int((7*(year + int((month+9)/12)))/4) + int((275*month)/9) + day + 1721013.5
    return J0 + UT/24

# function to return the position vector from the data file
def r(data):
    return [data[2][0], data[2][1], data[2][2]]

# function to return the velocity vector from the data file
def rdot(data):
    return [data[2][3], data[2][4], data[2][5]]

# function to return the required right essential from the data file
def alpha(data):
    return data[1][0] + data[1][1]/60 + data[1][2]/3600

# function to return the declination
def delta(data):
    if data[1][3] > 0 or data[1][3] == 0:
        Dec = data[1][3] + data[1][4]/60 + data[1][5]/3600   
    else:
        Dec = data[1][3] - data[1][4]/60 - data[1][5]/3600
    return Dec


#Calculating the Lagrange Coefficients

def f(r2,tau):
    return 1-(mu/2)*(tau**2/r2**3)

def g(r2,tau):
    return tau - (mu/6)*(tau**3/r2**3)

## Orbiting body direction cosine vector:

The orbiting body direction cosine vector can be determined from the right ascension and declination



<img src="visual%20for%20cosine%20vector.png" alt="cosine vector" style="width: 500px;"/>

In [6]:
def rho_fun(alpha,delta):
    return [cos(alpha)*cos(delta), sin(alpha)*cos(delta), sin(delta)]

In [7]:
t1 = time(obs1)
t2 = time(obs2)
t3 = time(obs3)

al = []
al.append(alpha(obs1))
al.append(alpha(obs2))
al.append(alpha(obs3))

de = []
de.append(delta(obs1))
de.append(delta(obs2))
de.append(delta(obs3))

R = []
R.append(r(obs1))
R.append(r(obs2))
R.append(r(obs3))

Rdot = []
Rdot.append(rdot(obs1))
Rdot.append(rdot(obs2))
Rdot.append(rdot(obs3))

In [8]:
rho = []
for i in range(3):
    rho.append(rho_fun(al[i],de[i]))

Calculate time intervals:

In [9]:
# subtracting the time intervals between observations
# here k is a constant called the Gaussian gravitational constant, expressed in radians per day
tau1 = k*(t1 - t2)
tau3 = k*(t3 - t2)
tau = k*(t3 - t1)

In [10]:
# some useful functions

def cross(a,b):
    return [a[1]*b[2] - a[2]*b[1], a[2]*b[0] - a[0]*b[2], a[0]*b[1] - a[1]*b[0]]

def dot(a,b):
    return a[0]*b[0]+a[1]*b[1]+a[2]*b[2]

def mul(k,a):
    return [k*a[0], k*a[1], k*a[2]]

def add(a,b):
    return [a[0]+b[0], a[1]+b[1], a[2]+b[2]]

def subtract(a,b):
    return [a[0]-b[0], a[1]-b[1], a[2]-b[2]]

def mag(a):
    return sqrt(a[0]**2 + a[1]**2 + a[2]**2)

Calculating cross products of the observational unit direction:
<img src="cross%20product.png" alt="cross product" style="width: 200px;"/>

In [11]:
P = []
P.append(cross(rho[1], rho[2]))
P.append(cross(rho[0], rho[2]))
P.append(cross(rho[0], rho[1]))

Calculating common scalar quantity (scalar triple product), by taking the dot product of the first observational unit vector with the cross product of the second and third observational unit vector:

<img src="scalar%20pr.png" alt="scalar product" style="width: 250px;"/>

In [12]:
# common scalar triple product
D0 = dot(rho[0],P[0])

# scalar quantities
D = [[],[],[]]
for i in range(3):
    for j in range(3):
        D[i].append(dot(R[i],P[j]))

Calculate scalar position coefficients using the formullas:

<img src="position%20coefficients%20formullas.png" alt="scalar product" style="width: 350px;"/>


In [13]:
# scalar position coefficients

A = (1/D0)*((-1)*D[0][1]*(tau3/tau) + D[1][1] + D[2][1]*(tau1/tau))
B = (1/(6*D0))*(D[0][1]*(tau3**2 - tau**2)*(tau3/tau) + D[2][1]*(tau**2 - tau1**2)*(tau1/tau))
E = dot(R[1], rho[1])

Calculate the coefficients of the scalar distance polynomial for the second observation of the orbiting body:
<img src="scalar%20distance%20polynomial%20.png" alt="scalar product" style="width: 700px;"/>


In [14]:
# scalar distance polynomial coefficients

a = (-1)*(A**2 + A*E + dot(R[2],R[2]))
b = (-2)*mu*B*(A+E)
c = (-1)*(mu**2)*(B**2)

# mu = gravitational parameter

In [15]:
#Newton Raphson Method to determine r2
#reducing error..

def func(x):
    return x**8 + a*(x**6) + b*(x**3) + c

def derivfunc(x):
    return 8*(x**7) + 6*a*(x**5) + 3*b*(x**2)

def newton(x):
    h = func(x)/derivfunc(x)
    while abs(h) >= 0.0001:
        h = func(x)/derivfunc(x)
        x = x - h
    return x

r2 = newton(-10) # +10

Calculating the slant range, the distance from the observer point to the orbiting body at their respective time:

<img src="slant%20range.png" alt="scalar product" style="width: 700px;"/>


In [16]:
# slant range determination

slant = []
slant.append((1/D0) * ((6*(r2**3)*(D[2][0]*(tau1/tau3)+D[1][0]*(tau/tau3)) + mu*D[2][0]*(tau**2-tau1**2)*(tau1/tau3))/(6*(r2**3) + mu*(tau**2 - tau3**2)) - D[0][0]))
slant.append(A + (mu*B)/(r2**3))
slant.append((1/D0) * ((6*(r2**3)*(D[0][2]*(tau3/tau1)-D[1][2]*(tau/tau1)) + mu*D[0][2]*(tau**2-tau3**2)*(tau3/tau1))/(6*(r2**3) + mu*(tau**2 - tau1**2)) - D[2][2]))

In [17]:
# orbiting body position vectors

r = []
for i in range(3):
    r.append(add(R[i],mul(slant[i],rho[i])))

In [18]:
# velocity vector

k = f(r2,tau1)*g(r2,tau3) - f(r2,tau3)*g(r2,tau1)
v2 = mul(1/k,subtract(mul(f(r2,tau1),r[2]),mul(f(r2,tau3),r[0])))

In [19]:
# function to intergrate all the above parameters and return the result

def elements(r,v):
    
    # orbital angluar momentum
    h = cross(r,v)
          
    # semi-major axis
    a = 1/(2/mag(r) - (mag(v)**2)/mu)
    print("The semi-major axis of the orbit : ", a)
    
    # eccentricity
    e = subtract(mul(1/mu,cross(v,h)), mul(1/mag(r),r))
    print("The eccentricity of the orbit : ", mag(e))
          
    # inclination
    print("The inclination of the orbit : ", acos(h[2]/mag(h))*(180/pi))
    
    # longitude of ascending node
    K = [0,0,1]
    I = [1,0,0]
    n = cross(K,h)
    nx = dot(I,n)
    if n[1] >= 0:
          omega = acos(nx/mag(n))
    else:
          omega = 2*pi - acos(nx/mag(n))
    print("The longitude of the ascending node : ", omega*(180/pi))
    
    # argument of perihilion
    if e[2] >= 0:
          arg = acos(dot(e,n)/(mag(n)*mag(e)))
    else:
          arg = 2*pi - acos(dot(e,n)/(mag(n)*mag(e)))
    print("The argument of perihelion : ", arg*(180/pi))
    
    # true anomaly
    tv = acos(dot(e,r)/(mag(e)*mag(r)))
    if dot(r,v) >= 0:
        tv = tv
    elif dot(r,v) < 0:
        tv = 2*pi - tv
    print("The true anomaly : ", tv*(180/pi))
    
    # type of the orbit:
    if mag(e)==0:
        print("Type of the orbit: Circle")
    elif mag(e)<1:
        print("Type of the orbit: Ellipse")
    elif mag(e)==1:
        print("Type of the orbit: Parabola")
    elif mag(e)>1:
        print("Type of the orbit: Hyperbola")
        
    

In [20]:
# when r2 = 14.388
elements(r[2],v2)

The semi-major axis of the orbit :  23.401379461985705
The eccentricity of the orbit :  0.9998857727328198
The inclination of the orbit :  94.36618550117838
The longitude of the ascending node :  187.39149469182553
The argument of perihelion :  344.23056324421765
The true anomaly :  179.3468760578816
Type of the orbit: Ellipse


In [21]:
# when r2 = -14.388
elements(r[2],v2)

The semi-major axis of the orbit :  23.401379461985705
The eccentricity of the orbit :  0.9998857727328198
The inclination of the orbit :  94.36618550117838
The longitude of the ascending node :  187.39149469182553
The argument of perihelion :  344.23056324421765
The true anomaly :  179.3468760578816
Type of the orbit: Ellipse
